In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
from statsmodels.stats.anova import AnovaRM
from scipy.stats import wilcoxon
from statsmodels.stats.multitest import multipletests

In [ ]:
# Helper Functions
def partial_eta_squared(aov):
    """Calculate partial eta-squared for ANOVA results."""
    aov = aov.copy()
    aov['pes'] = (aov['Num DF'] * aov['F Value']) / (aov['Num DF'] * aov['F Value'] + aov['Den DF'])
    return aov

def analyze_roi_data(data, experiment='exp1'):
    """Analyze ROI data with multiple comparison correction."""
    results = {}
    all_tests = []
    
    if experiment == 'exp1':
        conditions = ['object', 'scramble']
        rois = ['LO', 'pIPS']
    else:
        conditions = ['tool', 'nontool']
        rois = ['PFS', 'aIPS', 'LO', 'pIPS']
        
    for roi in rois:
        for hemi in ['left', 'right']:
            roi_data = data[(data['roi'] == roi) & (data['hemi'] == hemi)]
            cond1_data = roi_data[roi_data['cond'] == conditions[0]]['mean_act'].values
            cond2_data = roi_data[roi_data['cond'] == conditions[1]]['mean_act'].values
            
            # Calculate statistics
            t_stat, p_val = ttest_rel(cond1_data, cond2_data)
            mean_diff = np.mean(cond1_data - cond2_data)
            se_diff = np.std(cond1_data - cond2_data) / np.sqrt(len(cond1_data))
            d = mean_diff / np.sqrt((np.std(cond1_data)**2 + np.std(cond2_data)**2) / 2)
            
            # Store results
            all_tests.append({
                'roi': f'{hemi}_{roi}',
                'p_value': p_val,
                't_stat': t_stat,
                'mean_diff': mean_diff,
                'se': se_diff,
                'cohens_d': d,
                'n': len(cond1_data),
                f'mean_{conditions[0]}': np.mean(cond1_data),
                f'mean_{conditions[1]}': np.mean(cond2_data)
            })
    
    # Apply Holm-Bonferroni correction
    p_values = [test['p_value'] for test in all_tests]
    _, p_corrected, _, _ = multipletests(p_values, method='holm')
    
    # Update results with corrected p-values
    for test, p_corr in zip(all_tests, p_corrected):
        results[test['roi']] = test
        results[test['roi']]['p_corrected'] = p_corr
    
    return results

def run_rmanova(data, factors):
    """Run repeated measures ANOVA with effect sizes."""
    aov = AnovaRM(data=data, 
                  depvar='mean_act',
                  subject='sub',
                  within=factors).fit()
    
    aov_table = partial_eta_squared(aov.anova_table)
    return aov_table

In [ ]:
# Load Data
# Experiment 1
exp1_results = pd.read_csv("/user_data/csimmon2/git_repos/ptoc/results/selectivity/selectivity_summary.csv")

filtered_exp1 = exp1_results[(exp1_results['group'] == 'control') & 
                            (exp1_results['roi'].isin(['LO', 'pIPS'])) &
                            (exp1_results['sub'] != 'sub-084')]

# Experiment 2
exp2_results = pd.read_csv("/user_data/csimmon2/git_repos/ptoc/results/tools/selectivity/selectivity_summarytoolloc.csv")
filtered_exp2 = exp2_results[exp2_results['sub'].str.contains('spaceloc')]

In [ ]:
# FC Analysis (Experiment 1)
import os
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.multitest import fdrcorrection

# Import your parameters
curr_dir = f'/user_data/csimmon2/git_repos/ptoc'
study = 'ptoc'
study_dir = f"/lab_data/behrmannlab/vlad/{study}"
results_dir = '/user_data/csimmon2/git_repos/ptoc/results'

# Load data
data_df = pd.read_csv(f"{results_dir}/fc_ppi/matchvlad_seed_target_activations.csv")
fc_data = data_df[data_df['analysis'] == 'fc']

# Compare hemispheric patterns

# Ispilateral vs Contralateral
diff = ipsi.values - contra.values  # Convert to numpy arrays for element-wise subtraction
std_diff = np.std(diff, ddof=1) # Check for variance in differences
if std_diff == 0:  # Handle case where standard deviation is zero
    print("Standard deviation of differences is zero, preventing computation of Cohen's d.")
    d_ipsi = "NaN (variance is zero)"
else:
    # Compute Cohen's d for paired data
    d_ipsi = np.mean(diff) / std_diff

# Left vs Right pIPS seeds
left_pips = fc_data[fc_data['seed_hemisphere'] == 'left']['mean_activation']
right_pips = fc_data[fc_data['seed_hemisphere'] == 'right']['mean_activation']
w_stat_seed, p_val_seed = stats.wilcoxon(left_pips, right_pips)
# Independent effect size (Cohen's d for unpaired data)
d_seed = (np.mean(left_pips) - np.mean(right_pips)) / (
    np.sqrt((np.var(left_pips, ddof=1) + np.var(right_pips, ddof=1)) / 2) + 1e-10
)

# Left vs Right LO targets
left_lo = fc_data[fc_data['target_hemisphere'] == 'left']['mean_activation']
right_lo = fc_data[fc_data['target_hemisphere'] == 'right']['mean_activation']
w_stat_target, p_val_target = stats.wilcoxon(left_lo, right_lo)
# Independent effect size (Cohen's d for unpaired data)
d_target = (np.mean(left_lo) - np.mean(right_lo)) / (
    np.sqrt((np.var(left_lo, ddof=1) + np.var(right_lo, ddof=1)) / 2) + 1e-10
)

# Perform FDR correction for multiple comparisons
p_values = [p_val_ipsi, p_val_seed, p_val_target]
_, p_values_corrected = fdrcorrection(p_values)

# Print results
print("\nHemispheric Connectivity Comparisons:")

print(f"Ipsilateral vs Contralateral:")
print(f"W = {w_stat_ipsi}, p = {p_val_ipsi:.3f}, d = {d_ipsi}")

print(f"\nLeft vs Right pIPS seeds:")
print(f"W = {w_stat_seed}, p = {p_val_seed:.3f} (FDR-corrected p = {p_values_corrected[1]:.3f}), d = {d_seed:.3f}")
print(f"Mean Left = {np.mean(left_pips):.3f} ± {stats.sem(left_pips):.3f}")
print(f"Mean Right = {np.mean(right_pips):.3f} ± {stats.sem(right_pips):.3f}")

print(f"\nLeft vs Right LO targets:")
print(f"W = {w_stat_target}, p = {p_val_target:.3f} (FDR-corrected p = {p_values_corrected[2]:.3f}), d = {d_target:.3f}")
print(f"Mean Left = {np.mean(left_lo):.3f} ± {stats.sem(left_lo):.3f}")
print(f"Mean Right = {np.mean(right_lo):.3f} ± {stats.sem(right_lo):.3f}")


In [ ]:
# plot fc data exp 1
import matplotlib.pyplot as plt
import numpy as np

def create_fc_plot(fc_data):
    # Set style params
    plt.style.use('default')
    
    # Create figure with extra space at bottom for legend
    fig, ax = plt.subplots(figsize=(6, 7))
    
    # Define colors and markers
    colors = {'left': '#31688E',  # blue
             'right': '#7E4E90'}  # purple
    markers = {'left': 'o',  # circle
              'right': 's'}  # square
    
    # Calculate means and SEs for each connection
    connections = []
    means = []
    ses = []
    plot_colors = []
    plot_markers = []
    
    for seed_hemi in ['left', 'right']:
        for target_hemi in ['left', 'right']:
            curr_data = fc_data[
                (fc_data['seed_hemisphere'] == seed_hemi) &
                (fc_data['target_hemisphere'] == target_hemi)
            ]['mean_activation']
            connections.append(f"{seed_hemi}\npIPS")
            means.append(np.mean(curr_data))
            ses.append(np.std(curr_data) / np.sqrt(len(curr_data)))
            plot_colors.append(colors[seed_hemi])
            plot_markers.append(markers[target_hemi])
    
    # Plot
    x = np.arange(len(connections))
    
    # Error bars and points
    for i, (mean, se, color, marker) in enumerate(zip(means, ses, plot_colors, plot_markers)):
        plt.errorbar(i, mean, yerr=se, fmt='none', color=color,
                    capsize=5, capthick=1, elinewidth=2)
        plt.scatter(i, mean, color=color, marker=marker, s=100,
                   edgecolor='white', linewidth=1, zorder=3)
    
    # Create legend elements
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor=colors['left'],
               label='Left Seed', markersize=10, markeredgecolor='white'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor=colors['right'],
               label='Right Seed', markersize=10, markeredgecolor='white'),
        Line2D([0], [0], marker='o', color='black',
               label='Left Target', markersize=10, markerfacecolor='none'),
        Line2D([0], [0], marker='s', color='black',
               label='Right Target', markersize=10, markerfacecolor='none')
    ]
    
    # Add legend below the plot
    ax.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.15),
             ncol=2, frameon=False)
    
    # Styling
    plt.xticks(x, connections, rotation=30, ha='right')
    plt.ylabel('Functional Connectivity\n(Fisher Z)', fontsize=14)
    plt.xlabel('Connection', fontsize=14)
    
    # Set y-axis minimum to 0 and fewer ticks
    ax.set_ylim(bottom=0, top=0.2)
    ax.set_yticks(np.arange(0, 0.21, 0.05))  # Set ticks at 0, 0.05, 0.10, 0.15, 0.20
    
    # Theme
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.tick_params(axis='both', which='major', labelsize=12)
    
    plt.tight_layout()
    return fig

# Run plot
plot = create_fc_plot(fc_data)
plt.show()

In [ ]:
# FC Figure

# Functional Connectivity Group Averages - Inflated Surface Maps
### NOTE THAT LEFT SHOWS LEFT SEED TO LEFT HEMISPHERE RIGHT SHOWS RIGHT SEED TO RIGHT HEMISPHERE
import os
import numpy as np
import pandas as pd
import nibabel as nib
from nilearn import image, plotting, datasets, surface
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

def plot_inflated_brains(out_dir, rois, hemispheres):
    # Get fsaverage surface data
    fsaverage = datasets.fetch_surf_fsaverage()
    
    # First pass to determine global max across all ROIs
    global_vmax = 0
    for roi in rois:
        for analysis_type in ['fc']:  
            for hemi in hemispheres:
                img_file = f'{out_dir}/{roi}_{hemi}_{analysis_type}_avg.nii.gz'
                if os.path.exists(img_file):
                    img = nib.load(img_file)
                    data = img.get_fdata()
                    global_vmax = max(global_vmax, np.abs(data).max())

    # Create a single figure for all views
    fig = plt.figure(figsize=(5.5, 5))
    
    # Create 2x2 grid of 3D axes
    ax1 = fig.add_subplot(221, projection='3d')  # Top left - pIPS left lateral
    ax2 = fig.add_subplot(222, projection='3d')  # Top right - pIPS right lateral
    ax3 = fig.add_subplot(223, projection='3d')  # Bottom left - LO left lateral
    ax4 = fig.add_subplot(224, projection='3d')  # Bottom right - LO right lateral
    
    # Add row titles
    fig.text(0.5, 0.92, 'pIPS FC', va='center', ha='center', fontsize=12)
    fig.text(0.5, 0.42, 'LO FC', va='center', ha='center', fontsize=12)

    '''
    # Set titles
    ax1.set_title('pIPS left', pad=0)  # Reduced padding to match original
    ax2.set_title('pIPS right', pad=0)
    ax3.set_title('LO left', pad=0)
    ax4.set_title('LO right', pad=0)
    '''
    
    # Process each ROI and hemisphere
    for roi_idx, roi in enumerate(rois):
        for hemi_idx, hemi in enumerate(hemispheres):
            img_file = f'{out_dir}/{roi}_{hemi}_fc_avg.nii.gz'
            if os.path.exists(img_file):
                img = nib.load(img_file)
                surf_data = surface.vol_to_surf(img, fsaverage.pial_left if hemi == 'left' else fsaverage.pial_right)
                
                # Determine which axis to use
                ax = [ax1, ax2, ax3, ax4][roi_idx * 2 + hemi_idx]
                
                # Plot lateral view
                plotting.plot_surf_stat_map(
                    surf_mesh=fsaverage.infl_left if hemi == 'left' else fsaverage.infl_right,
                    stat_map=surf_data,
                    hemi=hemi,
                    view='lateral',
                    bg_map=fsaverage.sulc_left if hemi == 'left' else fsaverage.sulc_right,
                    colorbar=False, #title=f'{roi} {hemi} {analysis_type.upper()}',
                    cmap='RdYlBu_r',
                    axes=ax,
                    threshold=0.12,
                    vmax=global_vmax
                )
    
    # Add colorbar at the bottom center
    norm = Normalize(vmin=-global_vmax, vmax=global_vmax)
    sm = ScalarMappable(cmap='RdYlBu_r', norm=norm)
    sm.set_array([])
    
    # Create a new axes for the colorbar at the bottom center
    cbar_ax = fig.add_axes([0.35, 0.05, 0.3, 0.02])  # [left, bottom, width, height]
    plt.colorbar(sm, cax=cbar_ax, orientation='horizontal', label='Correlation')
    cbar_ax.tick_params(labelsize=8)  # Smaller font size
    
    plt.tight_layout()
    plt.show()

def main():
    study = 'ptoc'
    study_dir = f"/lab_data/behrmannlab/vlad/{study}"
    results_dir = '/user_data/csimmon2/git_repos/ptoc/results'
    
    group_out_dir = f'{results_dir}/group_averages'
    os.makedirs(group_out_dir, exist_ok=True)
    
    rois = ['pIPS', 'LO']
    hemispheres = ['left', 'right']
    
    plot_inflated_brains(group_out_dir, rois, hemispheres)

if __name__ == "__main__":
    main()

In [ ]:
# FC Analysis (Experiment 2)

In [ ]:
# Combined PPI (Experiment 1)
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.multitest import fdrcorrection
import warnings
warnings.filterwarnings('ignore')

# Directories
results_dir = '/user_data/csimmon2/git_repos/ptoc/results'

def calculate_effect_size(data1, data2=None):
    """Calculate effect size for one-sample or two-sample data"""
    if data2 is None:  # one-sample
        return np.mean(data1) / (np.std(data1, ddof=1) + 1e-10)
    else:  # two-sample
        diff = np.mean(data1) - np.mean(data2)
        pooled_std = np.sqrt((np.var(data1, ddof=1) + np.var(data2, ddof=1)) / 2)
        return diff / (pooled_std + 1e-10)

def analyze_ppi_data_nonparametric(data_df):
    """Analyze PPI data using non-parametric tests"""
    results = {}
    
    # 1. Test each condition against 0 using Wilcoxon signed-rank test
    conditions = []
    for seed_hemi in ['left', 'right']:
        for target_hemi in ['left', 'right']:
            subset = data_df[
                (data_df['seed_hemisphere'] == seed_hemi) &
                (data_df['target_hemisphere'] == target_hemi)
            ]['mean_activation']
            
            w_stat, p_val = stats.wilcoxon(subset)
            d = calculate_effect_size(subset)
            
            conditions.append({
                'connection': f"{seed_hemi} pIPS - {target_hemi} LO",
                'W': w_stat,
                'p': p_val,
                'd': d,
                'mean': np.mean(subset),
                'sem': stats.sem(subset)
            })
    
    # Apply FDR correction to zero-tests
    conditions_df = pd.DataFrame(conditions)
    _, conditions_df['p_fdr'] = fdrcorrection(conditions_df['p'])
    results['conditions'] = conditions_df
    
    # 2. Friedman test for overall differences
    # Reshape data for Friedman test
    wide_data = pd.pivot_table(
        data_df,
        values='mean_activation',
        index='subject',  # Make sure your data has a subject column
        columns=['seed_hemisphere', 'target_hemisphere']
    )
    
    chi2, friedman_p = stats.friedmanchisquare(*[wide_data[col] for col in wide_data.columns])
    results['friedman'] = {
        'chi2': chi2,
        'p': friedman_p,
        'df': len(wide_data.columns) - 1
    }
    
    # 3. Post-hoc Wilcoxon signed-rank tests for hemispheric differences
    post_hoc_results = []
    
    # Seed hemisphere comparison (collapsing across target)
    left_pips = data_df[data_df['seed_hemisphere'] == 'left']['mean_activation']
    right_pips = data_df[data_df['seed_hemisphere'] == 'right']['mean_activation']
    w_stat_seed, p_val_seed = stats.wilcoxon(left_pips, right_pips)
    d_seed = calculate_effect_size(left_pips, right_pips)
    
    post_hoc_results.append({
        'comparison': 'Left pIPS vs Right pIPS',
        'W': w_stat_seed,
        'p_uncorrected': p_val_seed,
        'd': d_seed,
        'mean_diff': np.mean(left_pips) - np.mean(right_pips),
        'left_mean': np.mean(left_pips),
        'left_sem': stats.sem(left_pips),
        'right_mean': np.mean(right_pips),
        'right_sem': stats.sem(right_pips)
    })
    
    # Target hemisphere comparison (collapsing across seed)
    left_lo = data_df[data_df['target_hemisphere'] == 'left']['mean_activation']
    right_lo = data_df[data_df['target_hemisphere'] == 'right']['mean_activation']
    w_stat_target, p_val_target = stats.wilcoxon(left_lo, right_lo)
    d_target = calculate_effect_size(left_lo, right_lo)
    
    post_hoc_results.append({
        'comparison': 'Left LO vs Right LO',
        'W': w_stat_target,
        'p_uncorrected': p_val_target,
        'd': d_target,
        'mean_diff': np.mean(left_lo) - np.mean(right_lo),
        'left_mean': np.mean(left_lo),
        'left_sem': stats.sem(left_lo),
        'right_mean': np.mean(right_lo),
        'right_sem': stats.sem(right_lo)
    })
    
    # Apply FDR correction to post-hoc tests
    post_hoc_df = pd.DataFrame(post_hoc_results)
    _, post_hoc_df['p_fdr'] = fdrcorrection(post_hoc_df['p_uncorrected'])
    results['post_hoc'] = post_hoc_df
    
    return results

# Load and prepare data
data_df = pd.read_csv(f"{results_dir}/fc_ppi/matchvlad_seed_target_activations.csv")
ppi_data = data_df[data_df['analysis'] == 'ppi']

# Run analysis
results = analyze_ppi_data_nonparametric(ppi_data)

# Print results
print("\nFriedman Test Results:")
print(f"χ²({results['friedman']['df']}) = {results['friedman']['chi2']:.3f}, p = {results['friedman']['p']:.3f}")

print("\nCondition Statistics (testing against 0):")
print(results['conditions'].round(3))

print("\nPost-hoc Tests:")
print(results['post_hoc'].round(3))

In [ ]:
# Plot PPI data exp 1
import numpy as np

def create_ppi_plot(ppi_data):
    # Set style params
    plt.style.use('default')
    
    # Create figure with extra space for legend
    fig, ax = plt.subplots(figsize=(6, 7))
    
    # Define colors and markers
    colors = {'left': '#31688E',  # blue
             'right': '#7E4E90'}  # purple
    markers = {'left': 'o',  # circle
              'right': 's'}  # square
    
    # Calculate means and SEs for each connection
    connections = []
    means = []
    ses = []
    plot_colors = []
    plot_markers = []
    
    for seed_hemi in ['left', 'right']:
        for target_hemi in ['left', 'right']:
            curr_data = ppi_data[
                (ppi_data['seed_hemisphere'] == seed_hemi) &
                (ppi_data['target_hemisphere'] == target_hemi) &
                (ppi_data['analysis'] == 'ppi')
            ]['mean_activation']
            connections.append(f"{seed_hemi}\npIPS")
            means.append(np.mean(curr_data))
            ses.append(np.std(curr_data) / np.sqrt(len(curr_data)))
            plot_colors.append(colors[seed_hemi])
            plot_markers.append(markers[target_hemi])
    
    # Plot
    x = np.arange(len(connections))
    
    # Error bars and points
    for i, (mean, se, color, marker) in enumerate(zip(means, ses, plot_colors, plot_markers)):
        plt.errorbar(i, mean, yerr=se, fmt='none', color=color,
                    capsize=5, capthick=1, elinewidth=2)
        plt.scatter(i, mean, color=color, marker=marker, s=100,
                   edgecolor='white', linewidth=1, zorder=3)
    
    # Create legend elements
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor=colors['left'],
               label='Left Seed', markersize=10, markeredgecolor='white'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor=colors['right'],
               label='Right Seed', markersize=10, markeredgecolor='white'),
        Line2D([0], [0], marker='o', color='black',
               label='Left Target', markersize=10, markerfacecolor='none'),
        Line2D([0], [0], marker='s', color='black',
               label='Right Target', markersize=10, markerfacecolor='none')
    ]
    
    # Add legend below the plot
    ax.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.15),
             ncol=2, frameon=False)
    
    # Styling
    plt.xticks(x, connections, rotation=30, ha='right')
    plt.ylabel('PPI Connectivity\n(Task Modulation)', fontsize=14)
    plt.xlabel('Connection', fontsize=14)
    
    # Set y-axis scale with tighter limits
    # Add just a bit of padding above and below the data
    data_range = max(means) - min(means)
    padding = data_range * 0.1  # 10% padding
    ymin = max(-0.05, min(means) - padding)  # Don't go below -0.05 if data doesn't warrant it
    ymax = max(means) + padding
    
    ax.set_ylim(ymin, ymax)
    ax.set_yticks(np.arange(np.floor(ymin*20)/20, ymax+0.02, 0.01))  # Ticks every 0.05
    
    # Theme
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.tick_params(axis='both', which='major', labelsize=12)
    
    plt.tight_layout()
    return fig

# Run plot
plot = create_ppi_plot(data_df)
plt.show()

In [ ]:
# Combined PPI (Experiment 2)
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.multitest import fdrcorrection
import warnings
warnings.filterwarnings('ignore')

# Directories
curr_dir = f'/user_data/csimmon2/git_repos/ptoc'
output_dir = f'{curr_dir}/tools'
results_dir = '/user_data/csimmon2/git_repos/ptoc/results'

def calculate_effect_size(data1, data2=None):
    """Calculate effect size for one-sample or two-sample data"""
    if data2 is None:  # one-sample
        return np.mean(data1) / (np.std(data1, ddof=1) + 1e-10)
    else:  # two-sample
        diff = np.mean(data1) - np.mean(data2)
        pooled_std = np.sqrt((np.var(data1, ddof=1) + np.var(data2, ddof=1)) / 2)
        return diff / (pooled_std + 1e-10)

def analyze_ppi_data_nonparametric(data_df):
    """Analyze PPI data using non-parametric tests including condition as a factor"""
    results = {}
    
    # 1. Overall Friedman test including condition
    wide_data = pd.pivot_table(
        data_df,
        values='value',
        index='subject',
        columns=['condition', 'seed_roi', 'seed_hemi', 'target_hemi'],
        aggfunc='first'
    ).dropna()
    
    # Perform Friedman test on all columns (includes condition)
    chi2, friedman_p = stats.friedmanchisquare(*[wide_data[col].values for col in wide_data.columns])
    results['friedman'] = {
        'chi2': chi2,
        'p': friedman_p,
        'df': len(wide_data.columns) - 1,
        'n_subjects': len(wide_data),
        'n_conditions': len(wide_data.columns),
        'conditions': list(wide_data.columns)
    }
    
    # 2. Test each connection against 0 (for both conditions)
    zero_test_results = []
    for condition in ['Tools', 'Nontools']:
        for seed_roi in ['pIPS', 'aIPS']:
            for seed_hemi in ['left', 'right']:
                for target_hemi in ['left', 'right']:
                    subset = data_df[
                        (data_df['condition'] == condition) &
                        (data_df['seed_roi'] == seed_roi) &
                        (data_df['seed_hemi'] == seed_hemi) &
                        (data_df['target_hemi'] == target_hemi)
                    ]['value']
                    
                    w_stat, p_val = stats.wilcoxon(subset)
                    d = calculate_effect_size(subset)
                    
                    zero_test_results.append({
                        'condition': condition,
                        'connection': f"{seed_hemi} {seed_roi} - {target_hemi} LO",
                        'W': w_stat,
                        'p': p_val,
                        'd': d,
                        'mean': np.mean(subset),
                        'sem': stats.sem(subset)
                    })
    
    # Apply FDR correction to all zero-tests together
    zero_test_df = pd.DataFrame(zero_test_results)
    _, zero_test_df['p_fdr'] = fdrcorrection(zero_test_df['p'])
    results['zero_tests'] = zero_test_df
    
    # 3. Compare Tools vs Nontools for each connection
    condition_comparisons = []
    for seed_roi in ['pIPS', 'aIPS']:
        for seed_hemi in ['left', 'right']:
            for target_hemi in ['left', 'right']:
                tools_data = data_df[
                    (data_df['condition'] == 'Tools') &
                    (data_df['seed_roi'] == seed_roi) &
                    (data_df['seed_hemi'] == seed_hemi) &
                    (data_df['target_hemi'] == target_hemi)
                ]['value']
                
                nontools_data = data_df[
                    (data_df['condition'] == 'Nontools') &
                    (data_df['seed_roi'] == seed_roi) &
                    (data_df['seed_hemi'] == seed_hemi) &
                    (data_df['target_hemi'] == target_hemi)
                ]['value']
                
                w_stat, p_val = stats.wilcoxon(tools_data, nontools_data)
                d = calculate_effect_size(tools_data, nontools_data)
                
                condition_comparisons.append({
                    'connection': f"{seed_hemi} {seed_roi} - {target_hemi} LO",
                    'W': w_stat,
                    'p': p_val,
                    'd': d,
                    'mean_diff': np.mean(tools_data) - np.mean(nontools_data),
                    'tools_mean': np.mean(tools_data),
                    'tools_sem': stats.sem(tools_data),
                    'nontools_mean': np.mean(nontools_data),
                    'nontools_sem': stats.sem(nontools_data)
                })
    
    # Apply FDR correction to condition comparisons
    condition_comp_df = pd.DataFrame(condition_comparisons)
    _, condition_comp_df['p_fdr'] = fdrcorrection(condition_comp_df['p'])
    results['condition_comparisons'] = condition_comp_df
    
    return results

# Load data
tools_df = pd.read_csv(f"{output_dir}/roi_ppi_sphere_summary_tools.csv")
nontools_df = pd.read_csv(f"{output_dir}/roi_ppi_sphere_summary_nontools.csv")

# Add condition labels
tools_df['condition'] = 'Tools'
nontools_df['condition'] = 'Nontools'
exp2_df = pd.concat([tools_df, nontools_df])

# Run analysis
results = analyze_ppi_data_nonparametric(exp2_df)

# Save and print results
with open(f"{results_dir}/exp2_ppi_stats_summary.txt", 'w') as f:
    # Overall test
    f.write(f"Overall Friedman Test Results:\n")
    f.write(f"χ²({results['friedman']['df']}) = {results['friedman']['chi2']:.3f}, p = {results['friedman']['p']:.3f}\n")
    f.write(f"Number of subjects: {results['friedman']['n_subjects']}\n")
    f.write(f"Number of conditions: {results['friedman']['n_conditions']}\n\n")
    
    # Zero tests by condition
    f.write("Tests against zero:\n")
    f.write(results['zero_tests'].round(3).to_string())
    f.write("\n\n")
    
    # Condition comparisons
    f.write("Tools vs Nontools Comparisons:\n")
    f.write(results['condition_comparisons'].round(3).to_string())

# Print results to console
print(f"\nOverall Friedman Test Results:")
print(f"χ²({results['friedman']['df']}) = {results['friedman']['chi2']:.3f}, p = {results['friedman']['p']:.3f}")
print(f"Number of subjects: {results['friedman']['n_subjects']}")
print(f"Number of conditions: {results['friedman']['n_conditions']}\n")

print("\nTests against zero:")
print(results['zero_tests'].round(3))

print("\nTools vs Nontools Comparisons:")
print(results['condition_comparisons'].round(3))

GCA

In [ ]:
# exp 1 GCA
# nonparametric gca univariate analysis
import numpy as np
import pandas as pd
from statsmodels.stats.multitest import fdrcorrection

# File paths
base_dir = '/user_data/csimmon2/git_repos/ptoc'
results_dir = f"{base_dir}/results"
fig_dir = f"{results_dir}/gca"

# Read the existing summary table
summary_table_combined = pd.read_csv(f"{fig_dir}/gca_summary_table_combined.csv")

# Apply FDR correction to the Wilcoxon p-values
p_values = summary_table_combined['p_value'].values
rejected, p_corrected = fdrcorrection(p_values, alpha=0.05, method='indep')

# Update table with FDR results
summary_table_combined['p_value_fdr'] = p_corrected
summary_table_combined['significant_fdr'] = rejected

# Save updated results
summary_table_combined.to_csv(f"{fig_dir}/gca_summary_table_combined_fdr.csv", index=False)

# Simple Cohen's d calculation
def calculate_cohens_d(mean_diff, std_diff):
    """Calculate Cohen's d as simple standardized mean difference"""
    return abs(mean_diff / std_diff)

print("\nCohen's d calculations:")
for _, row in summary_table_combined.iterrows():
    d = calculate_cohens_d(row['Mean f_diff'], row['Std f_diff'])
    print(f"{row['origin']}-{row['target']} ({row['condition']}): d = {d:.2f}")

# Update and display results
print("\nAfter FDR correction:")
print(summary_table_combined)

# Extract comparison p-values from the summary table
object_lpips_llo = summary_table_combined[(summary_table_combined['condition'] == 'Object') & 
                                        (summary_table_combined['origin'] == 'lpIPS') & 
                                        (summary_table_combined['target'] == 'lLO')]['p_value_fdr'].values[0]

object_lpips_rlo = summary_table_combined[(summary_table_combined['condition'] == 'Object') & 
                                        (summary_table_combined['origin'] == 'lpIPS') & 
                                        (summary_table_combined['target'] == 'rLO')]['p_value_fdr'].values[0]

scramble_lpips_llo = summary_table_combined[(summary_table_combined['condition'] == 'Scramble') & 
                                          (summary_table_combined['origin'] == 'lpIPS') & 
                                          (summary_table_combined['target'] == 'lLO')]['p_value_fdr'].values[0]

scramble_lpips_rlo = summary_table_combined[(summary_table_combined['condition'] == 'Scramble') & 
                                          (summary_table_combined['origin'] == 'lpIPS') & 
                                          (summary_table_combined['target'] == 'rLO')]['p_value_fdr'].values[0]

# Save FDR-corrected Wilcoxon test results
with open(f"{fig_dir}/gca_roi_pair_combined_comparison_results_fdr.txt", 'w') as f:
    f.write("GCA ROI Pair Combined Comparison Results (Wilcoxon tests):\n\n")
    f.write("Object Condition:\n")
    f.write("lpIPS-lLO vs rpIPS-lLO:\n")
    f.write(f"p_fdr = {object_lpips_llo:.4f}\n")
    f.write("lpIPS-rLO vs rpIPS-rLO:\n")
    f.write(f"p_fdr = {object_lpips_rlo:.4f}\n")
    f.write("\nScramble Condition:\n")
    f.write("lpIPS-lLO vs rpIPS-lLO:\n")
    f.write(f"p_fdr = {scramble_lpips_llo:.4f}\n")
    f.write("lpIPS-rLO vs rpIPS-rLO:\n")
    f.write(f"p_fdr = {scramble_lpips_rlo:.4f}\n")